In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chatgpt-sentiment-analysis/file.csv


In [3]:
!pip install transformers
!pip install datasets transformers huggingface_hub

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split

In [5]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["I like you", "I hate this movie"]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'POSITIVE', 'score': 0.9998695850372314},
 {'label': 'NEGATIVE', 'score': 0.9996687173843384}]

In [6]:
import torch
torch.cuda.is_available()

True

In [7]:
df = pd.read_csv('/kaggle/input/chatgpt-sentiment-analysis/file.csv')

In [8]:
df.head()

,Unnamed: 0,tweets,labels
0,0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,1,"Try talking with ChatGPT, our new AI system wh...",good
2,2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,3,"THRILLED to share that ChatGPT, our new model ...",good
4,4,"As of 2 minutes ago, @OpenAI released their ne...",bad


In [9]:
df.columns

Index(['Unnamed: 0', 'tweets', 'labels'], dtype='object')

In [10]:
df.drop(columns=['Unnamed: 0'], inplace=True)


In [11]:
df.columns

Index(['tweets', 'labels'], dtype='object')

In [12]:
df.shape

(219294, 2)

In [13]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

df['tweets'] = df['tweets'].apply(remove_html_tags)

df.head()

/tmp/ipykernel_30/4203560267.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
/tmp/ipykernel_30/4203560267.py:2: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


,tweets,labels
0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,"Try talking with ChatGPT, our new AI system wh...",good
2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,"THRILLED to share that ChatGPT, our new model ...",good
4,"As of 2 minutes ago, @OpenAI released their ne...",bad


In [14]:
from tqdm import tqdm

tqdm.pandas()

# Define punctuation and the function to remove punctuation
punctuation = string.punctuation

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', punctuation))

# Apply remove_punctuation function to 'tweets' column with tqdm progress
df['tweets'] = df['tweets'].progress_apply(remove_punctuation)

# Lowercasing the text with tqdm progress
df['tweets'] = df['tweets'].progress_apply(lambda x: x.lower())

# Display the modified DataFrame
df.head()

100%|██████████| 219294/219294 [00:00<00:00, 435826.92it/s]


,tweets,labels
0,chatgpt optimizing language models for dialogu...,neutral
1,try talking with chatgpt our new ai system whi...,good
2,chatgpt optimizing language models for dialogu...,neutral
3,thrilled to share that chatgpt our new model o...,good
4,as of 2 minutes ago openai released their new ...,bad


In [15]:
df['label'] = df['labels'].map({'neutral': 0, 'good': 1, 'bad': 2})  # Convert sentiments to binary labels
df = df[['label', 'tweets']]

In [16]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [17]:
dataset_split = dataset.train_test_split(test_size=0.3)

small_train_dataset = dataset_split['train'].shuffle(seed=42).select([i for i in range(20000)])

small_test_dataset = dataset_split['test'].shuffle(seed=42).select([i for i in range(5000)])

In [18]:
print("\nTraining dataset: \n")
print(small_train_dataset)

print("\nTraining dataset: \n")
print(small_test_dataset)


Training dataset: 

Dataset({
    features: ['label', 'tweets'],
    num_rows: 20000
})

Training dataset: 

Dataset({
    features: ['label', 'tweets'],
    num_rows: 5000
})


In [19]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
def preprocess_function(examples):
    return tokenizer(examples["tweets"], truncation=True, padding=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [21]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [22]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
!pip install evaluate

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00


In [24]:
# Import necessary libraries
import evaluate

# Load accuracy and F1 metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

# Update compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')  # Use 'weighted' for multi-class F1
    return {"accuracy": accuracy, "f1": f1}

In [25]:
from huggingface_hub import notebook_login
notebook_login()

In [26]:
from transformers import TrainingArguments 

repo_name = "chatgpt-model-on-sentiment-analysis"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch",
    push_to_hub = True
)

In [27]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [28]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,0.747200
1000,0.522800
1500,0.429200
2000,0.370300
2500,0.352300
3000,0.255600
3500,0.252300
4000,0.222500
4500,0.194800
5000,0.196000


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=6250, training_loss=0.3129090856933594, metrics={'train_runtime': 880.5462, 'train_samples_per_second': 113.566, 'train_steps_per_second': 7.098, 'total_flos': 2493610781515776.0, 'train_loss': 0.3129090856933594, 'epoch': 5.0})

In [29]:
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Trainer is attempting to log a value of "{'f1': 0.8691176083746959}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.5472574234008789,
 'eval_accuracy': 0.8696,
 'eval_f1': {'f1': 0.8691176083746959},
 'eval_runtime': 15.6371,
 'eval_samples_per_second': 319.752,
 'eval_steps_per_second': 20.016,
 'epoch': 5.0}

In [33]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

events.out.tfevents.1728457665.9a6430ad1243.30.1:   0%|          | 0.00/411 [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

events.out.tfevents.1728456769.9a6430ad1243.30.0:   0%|          | 0.00/7.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rahulbaba/chatgpt-model-on-sentiment-analysis/commit/fee5a22689486119a026de0ff17e6aec814a4e8a', commit_message='End of training', commit_description='', oid='fee5a22689486119a026de0ff17e6aec814a4e8a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rahulbaba/chatgpt-model-on-sentiment-analysis', endpoint='https://huggingface.co', repo_type='model', repo_id='rahulbaba/chatgpt-model-on-sentiment-analysis'), pr_revision=None, pr_num=None)

In [30]:
# save_directory = "/kaggle/working/sentiment_model_v1"

# trainer.save_model(save_directory)

In [34]:
from transformers import pipeline
sentiment_model = pipeline(model="rahulbaba/chatgpt-model-on-sentiment-analysis")
sentiment_model(["I like this move", "This movie is bad!"])

config.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_0', 'score': 0.9510214328765869},
 {'label': 'LABEL_2', 'score': 0.9976685643196106}]